In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
import requests
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords
nltk.download("stopwords")

In [ ]:
only_books = pd.read_csv("only_book.csv")
only_books.head(2)

,Unnamed: 0,Title,description,authors,publishedDate,ratingsCount,book_author,Summary,Language,Category,Publishers
0,0,The Impatient Gardener's Lawn Book,"Explains how to grow a green, beautiful lawn a...",['Jerry Baker'],1987,1.0,Jerry Baker,"Explains how to grow a green, beautiful lawn a...",en,['Gardening'],"Random House Digital, Inc."
1,14,Perry Mason in the Case of Too Many Murders,A Los Angeles businessman kills his dinner com...,['Thomas Chastain'],1990,2.0,Thomas Chastain,A Los Angeles businessman kills his dinner com...,en,['Fiction'],Avon Books


In [ ]:
merged_data = pd.read_csv("reviews1.csv")
merged_data.head(2)

,Title,description,authors,publishedDate,ratingsCount,book_author,Summary,Language,Category,Publishers,User_id,review/score,review/summary,review/text
0,The Impatient Gardener's Lawn Book,"Explains how to grow a green, beautiful lawn a...",['Jerry Baker'],1987,1.0,Jerry Baker,"Explains how to grow a green, beautiful lawn a...",en,['Gardening'],"Random House Digital, Inc.",A27H3RG5KF1MCI,4.0,"Pretty good, although a few flaws","This is, all-around, a pretty good lawn book. ..."
1,The Impatient Gardener's Lawn Book,"Explains how to grow a green, beautiful lawn a...",['Jerry Baker'],1987,1.0,Jerry Baker,"Explains how to grow a green, beautiful lawn a...",en,['Gardening'],"Random House Digital, Inc.",A2ISQPWVW0UYGW,5.0,Gardening Book,I bought this book as a present. They loved it...


In [ ]:
merged_data.drop(['description','authors','publishedDate','Language'],inplace=True,axis=1)
#merged_data.drop(['review/summary','review/text'],inplace=True,axis=1)
merged_data['Category'] = merged_data['Category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
merged_data.head(2)

,Title,ratingsCount,book_author,Summary,Category,Publishers,User_id,review/score,review/summary,review/text
0,The Impatient Gardener's Lawn Book,1.0,Jerry Baker,"Explains how to grow a green, beautiful lawn a...",Gardening,"Random House Digital, Inc.",A27H3RG5KF1MCI,4.0,"Pretty good, although a few flaws","This is, all-around, a pretty good lawn book. ..."
1,The Impatient Gardener's Lawn Book,1.0,Jerry Baker,"Explains how to grow a green, beautiful lawn a...",Gardening,"Random House Digital, Inc.",A2ISQPWVW0UYGW,5.0,Gardening Book,I bought this book as a present. They loved it...


In [ ]:
# FIltering 
x = merged_data.groupby('User_id').count()['review/score'] > 150
users = x[x].index
filtered_rating = merged_data[merged_data['User_id'].isin(users)]

y = filtered_rating.groupby('Title').count()['review/score']>=25
books = y[y].index
final_rating = filtered_rating[filtered_rating['Title'].isin(books)]



In [ ]:
final_rating.head(2)

,Title,ratingsCount,book_author,Summary,Category,Publishers,User_id,review/score,review/summary,review/text
1361,Hard Times,64.0,Charles Dickens,Edited with an Introduction and Notes by Kate ...,Fiction,Lulu.com,A3O2RCKAMSE9X7,5.0,Time Were Hard--And Dickens Shows How &amp; Why,It is unfortunate that HARD TIMES by Charles D...
1368,Hard Times,64.0,Charles Dickens,Edited with an Introduction and Notes by Kate ...,Fiction,Lulu.com,A319KYEIAZ3SON,4.0,"""Be in all things regulated and governed by fa...","Always concerned with issues of class, social ..."


In [ ]:
final_rating.shape

(18359, 10)

In [ ]:
pt = final_rating.pivot_table(index='Title',columns='User_id',values='review/score')


User-User Similarity

In [ ]:
for i in range(pt.shape[0]):
    avg=round(pt.iloc[i,:].mean(),2)
    pt.iloc[i,:]=pt.iloc[i,:].sub(avg)
pt.replace(np.nan,0,inplace=True)

In [ ]:

t2 = final_rating.iloc[:,[0,3]]
t2.drop_duplicates(inplace=True)


C:\Users\hp\AppData\Local\Temp\ipykernel_18780\3213997263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop_duplicates(inplace=True)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Sample book summaries

book_summary = list(t2['Summary'])

user_summary = "An epic fantasy with magical creatures and a quest to save the world..."

book_summary.append(user_summary)

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(book_summary)

cosine_similarities = linear_kernel(tfidf_matrix[-1], tfidf_matrix[:-1]).flatten()

t2['Sim'] = list(cosine_similarities)
t2

C:\Users\hp\AppData\Local\Temp\ipykernel_18780\4251986395.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2['Sim'] = list(cosine_similarities)


,Title,Summary,Sim
1361,Hard Times,Edited with an Introduction and Notes by Kate ...,0.000000
1633,Hard Times,This story of class conflict in Victorian Engl...,0.000000
1905,Hard Times,Yet this novel is not the cheerful celebration...,0.000000
2177,Hard Times,This edition includes new notes to the text. F...,0.000000
3523,One Hundred Years of Solitude,"The rise and fall, birth and death, of the myt...",0.000000
...,...,...,...
770408,Christine,Arnie Cunningham is in love with his car. Chri...,0.000000
770715,Christine,"A scarlet-and-white, 1958 Plymouth--salvaged, ...",0.000000
781241,The Lost World,Six years after the secret disaster at Jurassi...,0.000000
781377,The Lost World,The Lost World is a novel concerning an expedi...,0.148909


In [ ]:
t2.sort_values(by=['Sim'],ascending=False,inplace=True)

recommend = t2.iloc[:10,[0,2]]
recommend

C:\Users\hp\AppData\Local\Temp\ipykernel_18780\3294143097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.sort_values(by=['Sim'],ascending=False,inplace=True)


,Title,Sim
370760,The Hobbit,0.196475
781377,The Lost World,0.148909
157222,Shiloh,0.126337
407394,Alice's Adventures in Wonderland,0.119813
86651,Brave New World,0.105386
563848,Sister Carrie,0.102086
702907,Dandelion Wine,0.100208
101149,Lost Horizon,0.099236
558390,A Journey to the Center of the Earth,0.085293
692956,Jurassic Park,0.080499


In [ ]:
final_rating.head(3)


,Title,ratingsCount,book_author,Summary,Category,Publishers,User_id,review/score,review/summary,review/text
1361,Hard Times,64.0,Charles Dickens,Edited with an Introduction and Notes by Kate ...,Fiction,Lulu.com,A3O2RCKAMSE9X7,5.0,Time Were Hard--And Dickens Shows How &amp; Why,It is unfortunate that HARD TIMES by Charles D...
1368,Hard Times,64.0,Charles Dickens,Edited with an Introduction and Notes by Kate ...,Fiction,Lulu.com,A319KYEIAZ3SON,4.0,"""Be in all things regulated and governed by fa...","Always concerned with issues of class, social ..."
1376,Hard Times,64.0,Charles Dickens,Edited with an Introduction and Notes by Kate ...,Fiction,Lulu.com,A3ICRSCWVEZPAK,5.0,"Hard Times, plus an article by Thomas Carlyle!",I love all the Norton Critical editions of lit...


Genre

In [ ]:
genre = final_rating.iloc[:,[0,4,7]]
temp = list(set(list(final_rating.iloc[:,0])))

title = genre.groupby('Title')

In [ ]:
avg_rating_1 = []

for i in temp:
    t = title.get_group(i)
    rating = round(np.mean(list(t.iloc[:,-1])),1)
    avg_rating_1.append([t.iloc[0,0],t.iloc[0,1],rating])

In [ ]:
genre = pd.DataFrame(avg_rating_1,columns=['Title','Genre','Rating'])
genre.head(2)

,Title,Genre,Rating
0,House of Mirth,New York N Y,5.0
1,Mere Christianity,Religion,5.0


In [ ]:
author = final_rating.iloc[:,[0,2]]
author.drop_duplicates(inplace=True)
author.head(2)


C:\Users\hp\AppData\Local\Temp\ipykernel_18780\1591306427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author.drop_duplicates(inplace=True)


,Title,book_author
1361,Hard Times,Charles Dickens
3523,One Hundred Years of Solitude,Gabriel Garcia Marquez


In [ ]:
new_author = pd.merge(author, genre, on='Title', how='inner')
new_author.head(2)

,Title,book_author,Genre,Rating
0,Hard Times,Charles Dickens,Fiction,4.7
1,One Hundred Years of Solitude,Gabriel Garcia Marquez,Fiction,4.7


Author

In [ ]:
def author_recommend(author_name):
    
    data = new_author.groupby('book_author')
    authors = list(set(new_author.iloc[:,1]))
    if author_name in authors:
        t1 = data.get_group(author_name)
        t1.sort_values(by=['Rating'], inplace=True,ascending=False)
        recommend = t1.iloc[:5,0]
        return recommend
    else:
        return 0



In [ ]:
t = author_recommend('Charles Dickens')


C:\Users\hp\AppData\Local\Temp\ipykernel_18780\438176732.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1.sort_values(by=['Rating'], inplace=True,ascending=False)


0            Hard Times
9    Great Expectations
Name: Title, dtype: object